In [1]:
import numpy as np
import random
from tabulate import tabulate
from tqdm import tqdm
import ast

In [2]:
def generate_grid_world(length, width,path_lenght,holes_number,Random_State):
    
    random.seed(Random_State)
    #store all cells in a list
    Grid_Cells = []
    for row in range(length):
        for col in range(width):
            Grid_Cells.append([row,col])


    #specify the number of holes in the gridworld
    
    #specify the start point as a random cell
    start = [random.randint(0, length), random.randint(0, width)]

    #create a path from start point
    """instead of defining start and goal points,
      we define just a start point and a random path with a random lenght to
       another point and name it as goal point"""
    
    def random_path(Start, Path_Lenght,length, width):
        
        Path = []
        Path.append(Start)
        for i in range(Path_Lenght):
            
            #there are two moves that take us on a random cell named Goal [1,0], [0,1]
            
            move = random.choice([[1,0], [0,1]])
            
            #update the start cell/point by the above move
            Start = [x + y for x, y in zip(Start, move)]
            
            #if the movement take us out of our gridworld, we reverse the change in the start point
            if Start[0] < 0 or Start[1] < 0 or Start[0] > length-1 or Start[1] > width-1:

                Start = [x - y for x, y in zip(Start, move)]

            else:
                
                #create a path history
                Path.append(Start)

        Goal = Start

        return Goal,Path
    

    GoalPath = random_path(start, path_lenght,length, width)

    goal = GoalPath[0]
    path = GoalPath[1]

    #now we must eliminate the path cells from the Grid_Cells to choose hole cells from remaining cells

    FreeCells = [x for x in Grid_Cells if x not in path]

    Holes = random.sample(FreeCells, holes_number)

    #Also, we can visualize our gridworld in a simple way

    def mark_holes(holes):
        marked_data = [["Hole" if [row, col] in holes else [row, col] for col in range(width)] for row in range(length)]
        return marked_data
    
    marked_matrix = mark_holes(Holes)

    print(tabulate(marked_matrix, tablefmt="grid"))

    
    return length, width, start, goal, Holes, path,Grid_Cells

In [3]:
environment = generate_grid_world(5, 4,4,4,39)
environment

+--------+--------+--------+--------+
| Hole   | [0, 1] | [0, 2] | [0, 3] |
+--------+--------+--------+--------+
| [1, 0] | [1, 1] | [1, 2] | [1, 3] |
+--------+--------+--------+--------+
| Hole   | [2, 1] | [2, 2] | [2, 3] |
+--------+--------+--------+--------+
| Hole   | [3, 1] | Hole   | [3, 3] |
+--------+--------+--------+--------+
| [4, 0] | [4, 1] | [4, 2] | [4, 3] |
+--------+--------+--------+--------+


(5,
 4,
 [1, 2],
 [4, 3],
 [[2, 0], [3, 2], [3, 0], [0, 0]],
 [[1, 2], [1, 3], [2, 3], [3, 3], [4, 3]],
 [[0, 0],
  [0, 1],
  [0, 2],
  [0, 3],
  [1, 0],
  [1, 1],
  [1, 2],
  [1, 3],
  [2, 0],
  [2, 1],
  [2, 2],
  [2, 3],
  [3, 0],
  [3, 1],
  [3, 2],
  [3, 3],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3]])

In [57]:
def probability_distribution(grid_size,randomness):
    #random.seed(40)
    
    #by this function we generate probabilities which their sum is equal to 1
    def generate_probabilities(n):

        numbers = [random.random() for _ in range(n)]
        total_sum = sum(numbers)
        scaled_numbers = [num / total_sum for num in numbers]
        
        return scaled_numbers
    
    cells_prob = {}
    if randomness == 'stochastic':
        for cell in range(grid_size):
            
            #we set the number of probs to 4 due to 4 possible action for each cell (go to its neighbors)
            probs = generate_probabilities(4)

            cells_prob[cell] = probs
    elif randomness == 'equal probable':

        for cell in range(grid_size):

            cells_prob[cell] = [0.25,0.25,0.25,0.25]
    
    elif randomness == 'deterministic':
        for cell in range(grid_size):

            cells_prob[cell] = [0.03,0.06,0.01,0.9] #[0,0,0,1] ##[0.15,.15,0.1,0.6]


    #Note that we consider the correspondence between probabilities and actions as below:
    #probs = [p1, p2, p3, p4] ---> [[1,0],[-1,0],[0,1],[0,-1]]

    return cells_prob

def neighbor_cells(cell):

    grid_cells = environment[6]
    Actions = [[1,0],[-1,0],[0,1],[0,-1]]

    Neighbors = []
    Actions_Neighbors = []
    for action in Actions:

        neighbor = [x + y for x, y in zip(cell, action)]
        #if neighbor not in environment[4]:
        Neighbors.append(neighbor)
        Actions_Neighbors.append(action)

    return Neighbors, Actions_Neighbors

#Note
"""As we want to use monte carlo method for estimating the state values
   it has been assumed that we have not any knowledge about the environment.
   Therefore, we should consider the transitions into the holes cells
   (against the case of policy iteration)"""

def arbitrary_policy(randomness):
    #random.seed(randomness)
    
    policy = {}
    policy_action = {}
    for state in environment[6]:

        if state not in environment[4]:

            neighbors = neighbor_cells(state)[0]
            Actions_Neighbors = neighbor_cells(state)[1]

            allowed_positions = []

            for neighbor in neighbors:
                
                if neighbor in environment[6] and neighbor not in environment[4]:
                    
                    allowed_positions.append(neighbor)
        
            next_state = random.choice(allowed_positions)

            row = next_state[0] - state[0]
            col = next_state[1] - state[1]
            PolicyAction = [row, col]

            policy['{}'.format(state)] = next_state
            policy_action['{}'.format(state)] = PolicyAction


    return policy, policy_action

def state_reward(next_state):

    if next_state in environment[4]:

        r = -3
    
    elif next_state == environment[3]:

        r = 100
    
    elif next_state not in environment[6]:

        r = -2
    
    else:

        r = -1
    
    return r


state_indice_dict = {}
counter = 0
for state in environment[6]:

    state = str(state)
    state_indice_dict[state] = counter
    counter = counter + 1


def generate_trajectory(policy,randomness,environment_stochasticity):

    policy_action = policy[1]
    probs = probability_distribution(environment[0]*environment[1],environment_stochasticity)
    start = environment[2]
    terminate = start
    trajectory = [start]
    pure_trajectory = [start]
    c = 0
    while terminate != environment[3]:
        random.seed(randomness+c)
        Actions = [[1,0],[-1,0],[0,1],[0,-1]]
        action = policy_action[str(terminate)]
        Actions.remove(action)
        sorted_actions = Actions + [action]
        state_indice = state_indice_dict[str(terminate)]
        actions_prob = probs[state_indice]
        actions_prob.sort()

        selected_action = random.choices(sorted_actions, actions_prob)[0]
        
        next_state = [x + y for x, y in zip(terminate, selected_action)]
        pure_trajectory.append(next_state)
        
        #if the agent goes out of the gridworld, it stays in its current state
        if next_state not in environment[6]:
            next_state = terminate
        
        #if it drops into the holes, it goes to the start points
        elif next_state in environment[4]:
            next_state = start  

        terminate = next_state
        trajectory.append(terminate)
        c = c+1
    
    trajectory.append(environment[3])
    pure_trajectory.append(environment[3])

    return trajectory,pure_trajectory

## n-step TD for estimating $V \approx  v_{\pi}$

In [69]:
def n_step_TD(num_trials, n, policy, gamma, alpha, environment_stochasticity):

    V = {}
    for state in environment[6]:
    
        if state not in environment[4]:

            V[str(state)] = 0
    
    indice_state_dict = {}
    counter = 0
    for state in environment[6]:

        #state = str(state)
        indice_state_dict[counter] = state
        counter = counter + 1
    
    state_policy = policy[0]
    action_policy = policy[1]

    for trial in tqdm(range(num_trials)):

        TRAJECTORY = generate_trajectory(policy,trial,environment_stochasticity)

        trajectory = TRAJECTORY[0]
        pure_trajectory = TRAJECTORY[1]
        
        T = 20#float('inf')
        tau = 0
        t = -1
        while tau != T - 1:

            t = t + 1

            if t < T:
                
                #t_state = indice_state_dict[t]
                #next_state = state_policy[str(t_state)]
                

                if trajectory[t+1] == environment[3]:
                    T = t + 1
            
            tau = t - n + 1

            if tau >= 0:

                G = 0

                for i in range(tau+1, min(tau+n,T)+1):

                    r = state_reward(pure_trajectory[i+1])

                    G = G + (gamma ** (i-tau-1)) * r

                    if tau + n < T:

                        #print(tau , n)
                        
                        tau_n_state = indice_state_dict[tau + n]

                        if tau_n_state not in environment[4] and tau_n_state != environment[3]:

                            G = G + (gamma ** n) * V[str(tau_n_state)]
                    
                    tau_state = indice_state_dict[tau]
                    #tau
                    if tau_state not in environment[4]:
                        #print(type(tau_state))


                        V[str(tau_state)] = V[str(tau_state)] + alpha * (G - V[str(tau_state)])
            

    
    return V


In [71]:
policy_0 = arbitrary_policy(41)

n_step_TD(1000, 1, policy_0, 0.1, 0.9, 'deterministic')

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:11<00:00, 89.05it/s]


{'[0, 1]': -1.1111110251743115,
 '[0, 2]': -1.111110001929963,
 '[0, 3]': -1.1111000002880496,
 '[1, 0]': -1.1110000000242164,
 '[1, 1]': -1.1100000000031267,
 '[1, 2]': -1.1000000000002268,
 '[1, 3]': -1.000000000000018,
 '[2, 1]': -1.1100000000000005,
 '[2, 2]': -1.1,
 '[2, 3]': -1.0,
 '[3, 1]': -1.0,
 '[3, 3]': -1.111,
 '[4, 0]': -1.11,
 '[4, 1]': -1.1,
 '[4, 2]': -1.0,
 '[4, 3]': -1.0}

In [16]:
arbitrary_policy(1)[0]

{'[0, 1]': [0, 2],
 '[0, 2]': [0, 3],
 '[0, 3]': [1, 3],
 '[1, 0]': [1, 1],
 '[1, 1]': [1, 0],
 '[1, 2]': [1, 3],
 '[1, 3]': [1, 2],
 '[2, 1]': [1, 1],
 '[2, 2]': [2, 1],
 '[2, 3]': [3, 3],
 '[3, 1]': [4, 1],
 '[3, 3]': [4, 3],
 '[4, 0]': [4, 1],
 '[4, 1]': [4, 0],
 '[4, 2]': [4, 1],
 '[4, 3]': [4, 2]}

In [17]:
arbitrary_policy(11)[0]

{'[0, 1]': [0, 2],
 '[0, 2]': [0, 3],
 '[0, 3]': [0, 2],
 '[1, 0]': [1, 1],
 '[1, 1]': [0, 1],
 '[1, 2]': [1, 1],
 '[1, 3]': [2, 3],
 '[2, 1]': [3, 1],
 '[2, 2]': [2, 3],
 '[2, 3]': [2, 2],
 '[3, 1]': [2, 1],
 '[3, 3]': [4, 3],
 '[4, 0]': [4, 1],
 '[4, 1]': [4, 2],
 '[4, 2]': [4, 1],
 '[4, 3]': [3, 3]}

In [24]:
trj = generate_trajectory(arbitrary_policy(11),1,'stochastic')

for i in trj:

    if i == environment[2]:
        print(i)

[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
[1, 2]
